# Myers-Briggs Personality Type Prediction


## 1 - Packages ##

Let's first import all the packages that you will need.

In [203]:
# Package imports
import numpy as np
import tensorflow as tf

## 2 - Dataset ##

Let's get the dataset we will work on.

In [222]:
resource_location = "../data/mbti_1.csv" # location of the file
data = []

file = open(resource_location, 'r')
lines = file.readlines()[1:]

count = 0

for line in lines:
    personality_type, _, posts = line.partition(",")
    for post in posts.split("|||"):
        example = [post, personality_type]
        data.append(example)
        if(post.find("|||") != -1):
            print("hello")
    
data = np.asarray(data)
print(data.shape)


(422845, 2)


## 3 - Splite the dataset column wise ##

In [223]:
X, Y = np.hsplit(data,2)